<a href="https://colab.research.google.com/github/hselino/complaint_analysis/blob/main/summ_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nbformat

path = "/content/drive/MyDrive/Colab Notebooks/summ_fine_tune.ipynb"  # replace with your file

with open(path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove invalid widgets metadata
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

# Optionally, remove cell outputs too
for cell in nb.cells:
    if "outputs" in cell:
        cell["outputs"] = []
    if "execution_count" in cell:
        cell["execution_count"] = None

with open(path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("✅ Notebook cleaned and ready for GitHub.")


✅ Notebook cleaned and ready for GitHub.


In [ ]:
!pip install transformers[torch] datasets evaluate accelerate rouge_score nltk -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/prj/segmented_summ.csv')
df = df.dropna()
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "google/mt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = examples["content"]
    targets = examples["abstract"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]


    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/8995 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 2) for k, v in result.items()}

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

batch_size = 8

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
    fp16=False
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,

)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-037fca888085>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

trainer.save_model("/content/drive/MyDrive/prj/fine_tuned_model/summarizer")
tokenizer.save_pretrained("/content/drive/MyDrive/prj/fine_tuned_model/summarizer")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yeglkaan (yeglkaan-istanbul-k-lt-r-niversitesi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,3.106100,2.480863


Epoch,Training Loss,Validation Loss
1,3.106100,2.480863
2,3.266700,2.356499
3,2.968700,2.283521
4,2.975700,2.240071
5,2.889700,2.217304
6,2.696700,2.200676
7,2.639200,2.193352
8,2.695700,2.179108
9,2.498200,2.172052
10,2.507000,2.172003


('/content/drive/MyDrive/prj/fine_tuned_model/summarizer/tokenizer_config.json',
 '/content/drive/MyDrive/prj/fine_tuned_model/summarizer/special_tokens_map.json',
 '/content/drive/MyDrive/prj/fine_tuned_model/summarizer/spiece.model',
 '/content/drive/MyDrive/prj/fine_tuned_model/summarizer/added_tokens.json',
 '/content/drive/MyDrive/prj/fine_tuned_model/summarizer/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/prj/fine_tuned_model/summarizer")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/prj/fine_tuned_model/summarizer")

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

# Take a few examples from test set
examples = tokenized_datasets["test"].select(range(10))

predictions = []
references = []

for example in examples:
    input_text = tokenizer.decode(example["input_ids"], skip_special_tokens=True)
    label_text = tokenizer.decode(example["labels"], skip_special_tokens=True)
    output = model.generate(tokenizer(input_text, return_tensors="pt").input_ids)
    generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)

    predictions.append(generated_summary)
    references.append(label_text)

results = rouge.compute(predictions=predictions, references=references)
print(results)


ModuleNotFoundError: No module named 'evaluate'

In [ ]:
results

{'rouge1': np.float64(0.20488189441974955),
 'rouge2': np.float64(0.08134819557807174),
 'rougeL': np.float64(0.17565711005818263),
 'rougeLsum': np.float64(0.17622784205044606)}

In [ ]:
file_path = "asd.txt"

with open(file_path, "r", encoding="utf-8") as file:
    input_text = file.read()

print("Original Text:\n", input_text[:500])  # Preview first 500 chars

Original Text:
 Belgesellerde veya internet platformlarında dinozorlar, mamutlar ya da dodolar gibi nesli tükenmiş hayvanların etkileyici çizimlerini, üç boyutlu canlandırmalarını veya gerçekçi belgesel kesitlerini görmüş olabilirsiniz. Bu tasvirler hayvanların görünüşünü, yaşam biçimlerini ve çevreleriyle olan etkileşimlerini anlamamıza yardımcı olur. Ancak akla hemen şu soru gelir: Onlar hakkında bu kadar ayrıntılı bilgiye, özellikle de tüylerinin, pullarının,  derilerinin veya kürklerinin renklerine nasıl ul


In [ ]:
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    output = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

input_text

print("Summary:", generate_summary(input_text))

Summary: çok sayıda nesli tükenmiş hayvanların renklerini keşfetmek mümkün mü yoksa bunlar tamamen sanatçıların hayal gücüne mi dayanıyor?
